# Molecular Mechanics Parameters for a Protein-Ligand Complex

This exercise will familiarize you with code that builds a system from a PDB file and assigns force field parameters to proteins and small organic molecules. It is adapted from [talktorial 19 of TeachOpenCADD](https://github.com/volkamerlab/teachopencadd/tree/master/teachopencadd/talktorials/T019_md_simulation), a platform that aims to teach domain-specific skills and to provide pipeline templates as starting points for research projects.

When you are done with this exercise, save it under `Chem456-2022F/exercises` on Google Drive. It will be graded as satisfactory or unsatisfactory based on correctly completing the sections after `-->`. Do not remove the symbol `-->`.

# Part 0 - Setting up the required software

### Installation on Google Colab

The following code cells will install all required packages, if you are working on [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb). Installing the [condacolab](https://github.com/jaimergp/condacolab) package will restart the kernel.

In [ ]:
try:
    import google.colab
    !pip install condacolab
    import condacolab
    condacolab.install()
except ModuleNotFoundError:
    pass

In [ ]:
try:
    import condacolab
    from google.colab import files
    from IPython.display import clear_output
    condacolab.check()
    !conda install -q -y -c conda-forge mdtraj openmm cudatoolkit openmmforcefields openff-toolkit pdbfixer pypdb rdkit py3Dmol
except ModuleNotFoundError:
    on_colab = False
else:
    #check if installation was succesful
    try:
        import rdkit
        on_colab = True
        clear_output()  # clear the excessive installation outputs
        print("Dependencies successfully installed!")
    except ModuleNotFoundError:
        print("Error while installing dependencies!")

### Import dependencies

In [ ]:
import copy
from pathlib import Path

import requests
from IPython.display import display
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import mdtraj as md
import pdbfixer
import openmm as mm
import openmm.app as app
from openmm import unit
from openff.toolkit.topology import Molecule, Topology
from openmmforcefields.generators import GAFFTemplateGenerator

In [ ]:
# create data directory if not exists
HERE = Path(_dh[-1])
DATA = HERE / "data"
DATA.mkdir(exist_ok=True)

### EGFR kinase

The **E**pidermal **G**rowth **F**actor **R**eceptor (EGFR) is an important drug target with implications in cancer and inflammation ([Wikipedia](https://en.wikipedia.org/wiki/Epidermal_growth_factor_receptor)). It is a transmembrane protein with an extracellular receptor domain and an intracellular kinase domain. The binding of the endogenous ligand epidermal growth factor results in activation of the kinase domain via dimerization and autophosphorylation. The activated kinase domain can then phosphorylate downstream signaling proteins triggering DNA synthesis and cell proliferation ([_Cancers (Basel)_ (2017), **9**(5), 52](https://dx.doi.org/10.3390%2Fcancers9050052)). Inhibition of this kinase is the underlying mechanism of action of several already approved cancer drugs ([DrugBank](https://go.drugbank.com/bio_entities/BE0000767)). In this exercise, we will prepare the PDB structure **3POZ** of this kinase, which is in complex with the small molecule inhibitor **03P** ([PDB: 3POZ](https://www.rcsb.org/structure/3poz)).

# Part I - Preparing the protein-ligand complex

### Structure preparation

#### EGFR kinase

The **E**pidermal **G**rowth **F**actor **R**eceptor (EGFR) is an important drug target with implications in cancer and inflammation ([Wikipedia](https://en.wikipedia.org/wiki/Epidermal_growth_factor_receptor)). It is a transmembrane protein with an extracellular receptor domain and an intracellular kinase domain. The binding of the endogenous ligand epidermal growth factor results in activation of the kinase domain via dimerization and autophosphorylation. The activated kinase domain can then phosphorylate downstream signaling proteins triggering DNA synthesis and cell proliferation ([_Cancers (Basel)_ (2017), **9**(5), 52](https://dx.doi.org/10.3390%2Fcancers9050052)). Inhibition of this kinase is the underlying mechanism of action of several already approved cancer drugs ([DrugBank](https://go.drugbank.com/bio_entities/BE0000767)). In this exercise, we will prepare the PDB structure **3POZ** of this kinase, which is in complex with the small molecule inhibitor **03P** ([PDB: 3POZ](https://www.rcsb.org/structure/3poz)).

![3poz_assembly-1.jpeg](https://github.com/volkamerlab/teachopencadd/raw/ed3b2b6b655589d71355295af4c89363a63558b9/teachopencadd/talktorials/018_md_simulation/images/3poz_assembly-1.jpeg)

**Figure 4**: Structure 3POZ of the EGFR kinase domain bound to inhibitor 03P ([PDB: 3POZ](https://www.rcsb.org/structure/3poz)).

#### Download PDB file
The Protein Data Bank ([PDB](https://www.rcsb.org/)) allows for easy downloading of files via url.

In [ ]:
pdbid = "3POZ"
ligand_name = "03P"
pdb_path = DATA / f"{pdbid}.pdb"
pdb_url = f"https://files.rcsb.org/download/{pdbid}.pdb"

In [ ]:
r = requests.get(pdb_url)
r.raise_for_status()
with open(pdb_path, "wb") as f:
    f.write(r.content)

#### Protein preparation

A crucial part for successful simulation is a correct and complete system. Crystallographic structures retrieved from the Protein Data Bank often miss atoms, mainly hydrogens, and may contain non-standard residues. In this talktorial, we will use the Python package [PDBFixer](https://github.com/openmm/pdbfixer) to prepare the protein structure. However, co-crystallized ligands are not handled well by [PDBFixer](https://github.com/openmm/pdbfixer) and will thus be prepared separately.

In [ ]:
def prepare_protein(
    pdb_file, ignore_missing_residues=True, ignore_terminal_missing_residues=True, ph=7.0
):
    """
    Use pdbfixer to prepare the protein from a PDB file. Hetero atoms such as ligands are
    removed and non-standard residues replaced. Missing atoms to existing residues are added.
    Missing residues are ignored by default, but can be included.

    Parameters
    ----------
    pdb_file: pathlib.Path or str
        PDB file containing the system to simulate.
    ignore_missing_residues: bool, optional
        If missing residues should be ignored or built.
    ignore_terminal_missing_residues: bool, optional
        If missing residues at the beginning and the end of a chain should be ignored or built.
    ph: float, optional
        pH value used to determine protonation state of residues

    Returns
    -------
    fixer: pdbfixer.pdbfixer.PDBFixer
        Prepared protein system.
    """
    fixer = pdbfixer.PDBFixer(str(pdb_file))
    fixer.removeHeterogens()  # co-crystallized ligands are unknown to PDBFixer
    fixer.findMissingResidues()  # identify missing residues, needed for identification of missing atoms

    # if missing terminal residues shall be ignored, remove them from the dictionary
    if ignore_terminal_missing_residues:
        chains = list(fixer.topology.chains())
        keys = fixer.missingResidues.keys()
        for key in list(keys):
            chain = chains[key[0]]
            if key[1] == 0 or key[1] == len(list(chain.residues())):
                del fixer.missingResidues[key]

    # if all missing residues shall be ignored ignored, clear the dictionary
    if ignore_missing_residues:
        fixer.missingResidues = {}

    fixer.findNonstandardResidues()  # find non-standard residue
    fixer.replaceNonstandardResidues()  # replace non-standard residues with standard one
    fixer.findMissingAtoms()  # find missing heavy atoms
    fixer.addMissingAtoms()  # add missing atoms and residues
    fixer.addMissingHydrogens(ph)  # add missing hydrogens
    return fixer

In [ ]:
# prepare protein and build only missing non-terminal residues
prepared_protein = prepare_protein(pdb_path, ignore_missing_residues=False)

With everything automated, it is easy to take for granted what the program did. So we're going to check on the differences between the original model from the Protein Data Bank and the model from PDBFixer.

In [ ]:
help(prepared_protein.findMissingAtoms)
prepared_protein.missingAtoms

In [ ]:
help(prepared_protein.findMissingResidues)
prepared_protein.missingResidues

##### --> How many amino acids are missing from the crystal structure? How many atoms are missing from the other amino acids?

In [ ]:
import os
with open(os.path.join(DATA, f'{pdbid}-fixed.pdb'), "w") as pdb_file:  
  app.PDBFile.writeFile(
      prepared_protein.topology,
      prepared_protein.positions,
      file=pdb_file,
      keepIds=True,
  )

In [ ]:
import py3Dmol
view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(open(os.path.join(DATA, f'{pdbid}-fixed.pdb'), 'r').read(),'pdb')
view.addModel(open(os.path.join(DATA, f'{pdbid}.pdb'), 'r').read(),'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'purple'}})
view.setStyle({'model':1}, {'cartoon': {'color':'yellow', 'thickness':0.6}})
view.zoomTo()
view.show()

##### --> In the fixed model, do the added residues have any secondary structure? Do you think that the added residues could have secondary structure in solution?

#### Prepare ligand

After preparing the protein, we turn our attention to the ligand. Again, we need to add hydrogens, but also need to make sure the bond orders are correctly assigned, since some PDB entries may contain errors. We use the Python package [RDKit](https://github.com/rdkit/rdkit), an open source cheminformatics library.
We will provide the correct protonation state and bond orders to [RDKit](https://github.com/rdkit/rdkit) via a SMILES string. Uncharged isomeric SMILES strings for each co-crystallized ligand can be found in their respective [PDB](https://www.rcsb.org) entry. The ligand of PDB entry [3POZ](https://www.rcsb.org/structure/3poz) has the name [03P](https://www.rcsb.org/ligand/03P). If a ligand is likely to bind in its charged form or as a specific tautomer, such characteristics need to be incorporated into the SMILES string.

In [ ]:
def prepare_ligand(pdb_file, resname, smiles, depict=True):
    """
    Prepare a ligand from a PDB file via adding hydrogens and assigning bond orders. A depiction
    of the ligand before and after preparation is rendered in 2D to allow an inspection of the
    results. Huge thanks to @j-wags for the suggestion.

    Parameters
    ----------
    pdb_file: pathlib.PosixPath
       PDB file containing the ligand of interest.
    resname: str
        Three character residue name of the ligand.
    smiles : str
        SMILES string of the ligand informing about correct protonation and bond orders.
    depict: bool, optional
        show a 2D representation of the ligand

    Returns
    -------
    prepared_ligand: rdkit.Chem.rdchem.Mol
        Prepared ligand.
    """
    # split molecule
    rdkit_mol = Chem.MolFromPDBFile(str(pdb_file))
    rdkit_mol_split = Chem.rdmolops.SplitMolByPDBResidues(rdkit_mol)

    # extract the ligand and remove any already present hydrogens
    ligand = rdkit_mol_split[resname]
    ligand = Chem.RemoveHs(ligand)

    # assign bond orders from template
    reference_mol = Chem.MolFromSmiles(smiles)
    prepared_ligand = AllChem.AssignBondOrdersFromTemplate(reference_mol, ligand)
    prepared_ligand.AddConformer(ligand.GetConformer(0))

    # protonate ligand
    prepared_ligand = Chem.rdmolops.AddHs(prepared_ligand, addCoords=True)

    # 2D depiction
    if depict:
        ligand_2d = copy.deepcopy(ligand)
        prepared_ligand_2d = copy.deepcopy(prepared_ligand)
        AllChem.Compute2DCoords(ligand_2d)
        AllChem.Compute2DCoords(prepared_ligand_2d)
        display(
            Draw.MolsToGridImage(
                [ligand_2d, prepared_ligand_2d], molsPerRow=2, legends=["original", "prepared"]
            )
        )

    # return ligand
    return prepared_ligand

Calling this function with the isomeric SMILES string taken from the PDB entry for [03P](https://www.rcsb.org/ligand/03P) returns a correctly prepared ligand. Printed is a 2D-representation of the original and the prepared ligand for inspection.

In [ ]:
smiles = "CC(C)(O)CC(=O)NCCn1ccc2ncnc(Nc3ccc(Oc4cccc(c4)C(F)(F)F)c(Cl)c3)c12"
rdkit_ligand = prepare_ligand(pdb_path, ligand_name, smiles)

We will also show a comparison in 3D.

In [ ]:
original_PDB = open(os.path.join(DATA, f'{pdbid}.pdb'), 'r').read()
original_ligand_PDB = '\n'.join([line for line in original_PDB.split('\n') \
                        if line.find(ligand_name)==17])

In [ ]:
import py3Dmol
view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(rdkit.Chem.MolToMolBlock(rdkit_ligand),'sdf')
view.addModel(original_ligand_PDB,'pdb')
view.setStyle({'model':0}, {'stick': {}})
view.setStyle({'model':1}, {'sphere': {'scale':0.25, 'colorscheme':'yellowCarbon'}})
view.zoomTo()
view.show()

##### --> Compare and contrast the models of the ligand in rdkit and the original pdb file? What is similar and different about the atoms? What is similar and different about the 3D structure?

##### --> How do you expect the three-dimensional structure of the ligand in aqueous solution compare to the structure in the complex?

The ligand is in rdkit format, but we would like it to be in OpenMM format so that we can merge it with the protein. This next function will perform that task.

#### Merge protein and ligand

In the next step, we want to merge the prepared protein and ligand structures using the Python package [MDTraj](https://github.com/mdtraj/mdtraj). [MDTraj](https://github.com/mdtraj/mdtraj) can handle the prepared protein, which is currently a [PDBFixer](https://github.com/openmm/pdbfixer) molecule, a format that has a topology and atom positions similar to and usually interchangeable with [OpenMM Modeller](http://docs.openmm.org/latest/userguide/application.html#model-building-and-editing) topologies and positions. For the ligand however, we need to do several conversions, since it is currently an [RDKit](https://github.com/rdkit/rdkit) molecule.

In [ ]:
def rdkit_to_openmm(rdkit_mol, name="LIG"):
    """
    Convert an RDKit molecule to an OpenMM molecule.
    Inspired by @hannahbrucemcdonald and @glass-w.

    Parameters
    ----------
    rdkit_mol: rdkit.Chem.rdchem.Mol
        RDKit molecule to convert.
    name: str
        Molecule name.

    Returns
    -------
    omm_molecule: simtk.openmm.app.Modeller
        OpenMM modeller object holding the molecule of interest.
    """
    # convert RDKit to OpenFF
    off_mol = Molecule.from_rdkit(rdkit_mol)

    # add name for molecule
    off_mol.name = name

    # add names for atoms
    element_counter_dict = {}
    for off_atom, rdkit_atom in zip(off_mol.atoms, rdkit_mol.GetAtoms()):
        element = rdkit_atom.GetSymbol()
        if element in element_counter_dict.keys():
            element_counter_dict[element] += 1
        else:
            element_counter_dict[element] = 1
        off_atom.name = element + str(element_counter_dict[element])

    # convert from OpenFF to OpenMM
    off_mol_topology = off_mol.to_topology()
    mol_topology = off_mol_topology.to_openmm()
    mol_positions = off_mol.conformers[0]

    # convert units from Ångström to Nanometers
    for atom in mol_positions:
        coords = atom / atom.unit
        atom = (coords / 10.0) * unit.nanometers  # since openmm works in nm

    # combine topology and positions in modeller object
    omm_mol = app.Modeller(mol_topology, mol_positions)

    return omm_mol

In [ ]:
omm_ligand = rdkit_to_openmm(rdkit_ligand, ligand_name)

Now protein and ligand are both in [OpenMM](https://github.com/openmm/openmm) like formats and can be merged with [MDTraj](https://github.com/mdtraj/mdtraj).

In [ ]:
def merge_protein_and_ligand(protein, ligand):
    """
    Merge two OpenMM objects.

    Parameters
    ----------
    protein: pdbfixer.pdbfixer.PDBFixer
        Protein to merge.
    ligand: simtk.openmm.app.Modeller
        Ligand to merge.

    Returns
    -------
    complex_topology: simtk.openmm.app.topology.Topology
        The merged topology.
    complex_positions: simtk.unit.quantity.Quantity
        The merged positions.
    """
    # combine topologies
    md_protein_topology = md.Topology.from_openmm(protein.topology)  # using mdtraj for protein top
    md_ligand_topology = md.Topology.from_openmm(ligand.topology)  # using mdtraj for ligand top
    md_complex_topology = md_protein_topology.join(md_ligand_topology)  # add them together
    complex_topology = md_complex_topology.to_openmm()

    # combine positions
    total_atoms = len(protein.positions) + len(ligand.positions)

    # create an array for storing all atom positions as tupels containing a value and a unit
    # called OpenMM Quantities
    complex_positions = unit.Quantity(np.zeros([total_atoms, 3]), unit=unit.nanometers)
    complex_positions[: len(protein.positions)] = protein.positions  # add protein positions
    complex_positions[len(protein.positions) :] = ligand.positions  # add ligand positions

    return complex_topology, complex_positions

In [ ]:
complex_topology, complex_positions = merge_protein_and_ligand(prepared_protein, omm_ligand)
print("Complex topology has", complex_topology.getNumAtoms(), "atoms.")
# NBVAL_CHECK_OUTPUT

Let's visualize the complete model.

In [ ]:
import os
with open(os.path.join(DATA, f'complex.pdb'), "w") as pdb_file:  
  app.PDBFile.writeFile(
      complex_topology,
      complex_positions,
      file=pdb_file,
      keepIds=True,
  )

In [ ]:
import py3Dmol
view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(open(os.path.join(DATA, f'complex.pdb'), 'r').read(),'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'purple'}})
view.setStyle({'resn':ligand_name}, {'stick': {'colorscheme':'yellowCarbon'}})
view.addStyle({'within':{'distance':'7', 'sel':{'resn':ligand_name}}}, {'stick': {}})
view.zoomTo({'resn':ligand_name})
view.show()

### Force field setup

We can now use the coordinates of the complex to set up the molecular mechanics force field. 

#### Assigning force field parameters



Force fields describe the forces between atoms within and between molecules. They are parametric equations with components for different forces (bond stretching, van-der-Waals and more). The parameter values are usually derived experimentally and change for each MD scenario, depending on the molecules involved and the simulation settings. The result is a mathematical description of the energy landscape of the system, in which the forces acting on each particle result from the gradient of the potential energy with respect to the coordinates of the atoms.

Several force fields are available, each with its own characteristics ([_J Chem Inf Model_ (2018), **58**(3), 565-578](https://doi.org/10.1021/acs.jcim.8b00042)). In this notebook, we will use a member of the AMBER force field family, which are widely used for modeling proteins. Their functional form is:

$$V(r^N) = \sum_{i \in  bonds}k_{bi} (l_i-l^0_i)^2 + \sum_{i \in  angles}k_{ai}(\theta_i - \theta^0_i)^2 + \sum_{i\in torsions} \sum_n \frac{1}{2} V_i^n[1+cos(nw_i-\gamma_i)]$$
$$+ \sum_{j=1}^{N-1}\sum_{I=j+1}^{N} f_{ij}\in ij [(\frac{r^0_{ij}}{r_{ij}})^{12}-2(\frac{r^0_{ij}}{r_{ij}})^{6}]+\frac{q_iq_j}{4\pi \in_0 r_{ij}}$$

The formula consists of a sum of different components. The first three components contain information about bond lengths, angles and torsions (intramolecular forces). The last component describes intermolecular, non-bonded forces like van-der-Waals forces and electrostatic interactions. The various parameters, denoted by a superscript 0, depend on the force field used and vary between all members of the AMBER force field family. Note that these force fields assume fixed-charge particles and do not allow polarization, nor do they consider how a local charge influences its surroundings. 

The following visual representation of force fields components shows the same concepts in a more intuitive way.

![MM_PEF.png](https://github.com/volkamerlab/teachopencadd/raw/d1ded86bb2c82ef088cc5145d0bcb997f6eab7dd/teachopencadd/talktorials/018_md_simulation/images/MM_PEF.png)

**Figure 2**: Components of a molecular mechanics force field (Edboas via [Wikimedia](https://commons.wikimedia.org/w/index.php?curid=4194424)).

Common force fields like AMBER have parameters for amino acids, nucleic acids, water and ions and usually offer several options to choose from depending on your aim. We use the `amber14-all.xml` force field file, which is shipped with OpenMM and includes parameters for proteins, DNA, RNA and lipids. For solvation we use the standard three-site [water model](https://en.wikipedia.org/wiki/Water_model) [**TIP3P**](https://aip.scitation.org/doi/10.1063/1.445869).

Parameters for ligands however are not included. To generate these parameters, we can use the **G**eneral **A**MBER **F**orce**F**ield ([GAFF](http://ambermd.org/antechamber/gaff.html)), which is implemented in the Python package [OpenMM Forcefields](https://github.com/openmm/openmmforcefields). The following function generates a force field object holding standard AMBER parameters and additionally includes parameters for a small molecule if required.

In [ ]:
def generate_forcefield(
    rdkit_mol=None, protein_ff="amber14-all.xml", solvent_ff="amber14/tip3pfb.xml"
):
    """
    Generate an OpenMM Forcefield object and register a small molecule.

    Parameters
    ----------
    rdkit_mol: rdkit.Chem.rdchem.Mol
        Small molecule to register in the force field.
    protein_ff: string
        Name of the force field.
    solvent_ff: string
        Name of the solvent force field.

    Returns
    -------
    forcefield: simtk.openmm.app.Forcefield
        Forcefield with registered small molecule.
    """
    forcefield = app.ForceField(protein_ff, solvent_ff)

    if rdkit_mol is not None:
        gaff = GAFFTemplateGenerator(
            molecules=Molecule.from_rdkit(rdkit_mol, allow_undefined_stereo=True)
        )
        forcefield.registerTemplateGenerator(gaff.generator)

    return forcefield

In [ ]:
forcefield = generate_forcefield(rdkit_ligand)

#### Adding solvent

Often, molecular systems are simulated in a box filled with solvent such as water. These boxes are of finite size, which results in problems for molecules at or near the box boundaries. With which molecules should those interact? Periodic boundary conditions can avoid such boundary artifacts by simulating a theoretically infinite system. Molecules at one boundary of the box thereby interact with molecules at the boundary on the other side of the box. This mimics a situation, in which the simulation box is surrounded by replicas of itself. When visualizing such MD simulations, one can often observe that particles leave the box at one side (Fig. 3). However, they re-appear at the same time on the other side of the box with the same velocity. For simulations under periodic boundary conditions, it is recommended to use a simulation box large enough, so that the simulated macromolecule does not come into contact with neighboring images of itself.

![MD_water.gif](https://github.com/volkamerlab/teachopencadd/raw/d1ded86bb2c82ef088cc5145d0bcb997f6eab7dd/teachopencadd/talktorials/018_md_simulation/images/MD_water.gif)

**Figure 3**: Molecular dynamics simulation of water molecules with periodic boundary conditions (Kmckiern via [Wikimedia](https://commons.wikimedia.org/wiki/File:MD_water.gif)).

With our configured force field we can now  use the  [OpenMM Modeller](http://docs.openmm.org/latest/userguide/application.html#model-building-and-editing) class to create the MD environment, a simulation box which contains the complex and is filled with a solvent. The standard solvent is water with a specified amount of ions. The size of the box can be determined in various ways. We define it with a padding, which results in a cubic box with dimensions dependent on the largest dimension of the complex.

> Note this step can take a long time, in the order of minutes, depending on your hardware.

In [ ]:
modeller = app.Modeller(complex_topology, complex_positions)
modeller.addSolvent(forcefield, padding=1.0 * unit.nanometers, ionicStrength=0.15 * unit.molar)

Now let's visualize the complex in water. In this visualization, we look at the position of oxygen because py3Dmol does not keep hydrogens.

In [ ]:
import os
with open(os.path.join(DATA, f'complex-solvated.pdb'), "w") as pdb_file:  
  app.PDBFile.writeFile(
      modeller.topology,
      modeller.positions,
      file=pdb_file,
      keepIds=True,
  )

In [ ]:
import py3Dmol
view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(open(os.path.join(DATA, f'complex-solvated.pdb'), 'r').read(),'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'purple'}})
view.setStyle({'resn':ligand_name}, {'stick': {'colorscheme':'yellowCarbon'}})
view.addStyle({'resn':'HOH'},{'sphere':{'radius':'0.2'}})
view.zoomTo()
view.show()

##### --> In this system, how many particles are used to model water compared to the complex? Hint: Use the `len()` function for `complex_positions` and `modeller.positions`.

With our solvated system and force field, we can finally create an [OpenMM System](http://docs.openmm.org/development/api-python/generated/openmm.openmm.System.html#openmm.openmm.System). This is what we will work with in our molecular simulation lab.

In [ ]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME)
integrator = mm.LangevinIntegrator(
    300 * unit.kelvin, 1.0 / unit.picoseconds, 2.0 * unit.femtoseconds
)
simulation = app.Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

## Saving files

In [ ]:
import os

# Prepare directories
if not os.path.isdir('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

class_dir = os.path.join('/content','drive','MyDrive','Chem456-2022F')
if not os.path.isdir(class_dir):
  raise ValueError('Class directory not set up')
else:
  os.chdir(class_dir)

exercises_dir = os.path.join(class_dir,'exercises')
if not os.path.isdir(exercises_dir):
  raise ValueError('Exercises directory not set up')
else:
  os.chdir(exercises_dir)

os.chdir(exercises_dir)
system_preparation_dir = os.path.join(exercises_dir,'06-system_preparation')
if not os.path.isdir(system_preparation_dir):
  os.mkdir('06-system_preparation')

# Store files
with open(os.path.join(system_preparation_dir, 'complex-solv.xml'), "w") as xml_file:
  xml_file.write(mm.XmlSerializer.serialize(system))

with open(os.path.join(system_preparation_dir, 'complex-solv.pdb'), "w") as pdb_file:  
  app.PDBFile.writeFile(
      simulation.topology,
      simulation.context.getState(getPositions=True, enforcePeriodicBox=True).getPositions(),
      file=pdb_file,
      keepIds=True,
  )

# References

- Review on force fields ([_J Chem Inf Model_ (2018), **58**(3), 565-578](https://doi.org/10.1021/acs.jcim.8b00042))
- Review on EGFR in cancer ([_Cancers (Basel)_ (2017), **9**(5), 52](https://dx.doi.org/10.3390%2Fcancers9050052))
- Summarized statistical knowledge from Pierre-Simon Laplace ([Théorie Analytique des Probabilités _Gauthier-Villars_ (1820), **3**)](https://archive.org/details/uvrescompltesde31fragoog/page/n15/mode/2up)
- Inspired by a notebook form Jaime Rodríguez-Guerra ([github](https://github.com/jaimergp/uab-msc-bioinf/blob/master/MD%20Simulation%20and%20Analysis%20in%20a%20Notebook.ipynb))
- Repositories of [OpenMM](https://github.com/openmm/openmm) and [OpenMM Forcefields](https://github.com/openmm/openmmforcefields), [RDKit](https://github.com/rdkit/rdkit), [PyPDB](https://github.com/williamgilpin/pypdb), [MDTraj](https://github.com/mdtraj/mdtraj), [PDBFixer](https://github.com/openmm/pdbfixer)
- Wikipedia articles about [AMBER](https://en.wikipedia.org/wiki/AMBER) and [force fields](https://en.wikipedia.org/wiki/Force_field_(chemistry)) in general